# Create a TensorFlow Lite edge configuration package

In this notebook, the main goal is to create a pipeline with all of the contents that are necessary for the execution of the model on an Industrial Edge device.  
In order to put the elements together, this example collects files

from [10-CreateClassificationModel](10-CreateClassificationModel.ipynb) notebook:  
- **classification_mobilnet.tflite**: the trained model for classification with TFlite

from [20-CreateInferenceWrapper](20-CreateInferenceWrapper.ipynb) notebook:  
- **entrypoint.py**: the script that is called by the runtime to execute the model on the Edge side
- **payload.py**: contains the method which extracts the payload and create a PIL Image to be processed
- **vision_classifier.py**: contains the method to utilize the model and produces a prediction

Please note, there is no official TFLite runtime for Windows.

### Imports  

In [ ]:
from simaticai import deployment

### Create a single component

In this step, we create a `PythonComponent` that reads the input data `vision_payload` by `entrypoint.py`, processes the images by `vision_classifier.py` with model `classification_mobilnet.tflite` and produces an output `prediction` by `entrypoint.py` script.

In [ ]:
COMPONENT_DESCRIPTION ="""\
This component uses a trained TensorFlow Lite image classification model that reads the vision_payload input and produces a prediction as an output.
"""
INPUT_DESCRIPTION ="""
Vision connector MQTT payload holding the image to be classified.
"""
OUTPUT_DESCRIPTION ="""
The most probable class predicted for the image as an integer string.
"""

component = deployment.PythonComponent(
    name='inference', 
    python_version='3.11',
    desc=COMPONENT_DESCRIPTION)

component.add_resources('..', 'entrypoint.py')
component.set_entrypoint('entrypoint.py')
component.add_resources('..', ['src/payload.py', 'src/vision_classifier.py'])

component.add_input('vision_payload', 'ImageSet')
component.add_output('prediction', 'String')

#### Add metrics
It can be useful to monitor the pipeline, e.g. watch the prediction probabilities. The metric name must contain an underscore (`_`), because the part before the underscore is used to group custom metrics on the dashboard.

**&#9888; Remember!**
You have to use the same names here and in the inference wrapper script.

In [ ]:
component.add_metric("ic_probability")

### Add dependencies

All of the required dependencies are collected in the file `runtime_requirements.txt`. See [HOWTO](../HOWTO.md) for more possibilities.

In [ ]:
component.set_requirements('../runtime_requirements-py3.11.txt')

### Add a model

In [ ]:
component.add_resources('..', 'models/classification_mobilnet.tflite')

### Create a pipeline from this component

Now you can use the component to create a pipeline configuration.  

In [ ]:
PIPELINE_DESCRIPTION ="""\
This pipeline runs a TensorFlow Lite model on an Industrial Edge device.
The model was trained to recognize and classify images of following Siemens SIMATIC automation products: ET 200AL, ET 200eco PN, ET 200SP, S7-1200, S7-1500.

The pipeline is designed to be fed from Vision Connector via ZMQ in BayerRG8 pixel format.
The pipeline output is to be sent to the Databus.
"""

#To assure compatibility with older versions of AI SDK (<v1.5.0), you must define the `version` parameter in the `from_components()` method
pipeline = deployment.Pipeline.from_components([component], name='ImageClassification', desc=PIPELINE_DESCRIPTION)
pipeline

### Build the edge configuration package

This step creates the proper content in the defined target folder and creates the edge configuration package as a zip file.  
The export method first validates the pipeline and raises an error if it finds any problems. Manual validation is also possible with the `pipeline.validate()` method.

Edge packages are identified by their `package id` and `version` attributes, and will be grouped in AI Inference Server and in other Edge applications by `package id`.
When saving a pipeline - in the `export()` method - you can specify a `pacakge id` in a UUID 4 compliant format, or an automatically generated one will be assigned. 
If no `package id` is defined in the `export()` method, and AI SDK finds an already assigned `package id` in a previously generated, and similarly named package, the `package id` found in the latest package will be used.

AI SDK will automatically assign and increase the version number of a pipeline every time a package is saved, unless a new package id is assigned in the `export()` method or an explicit version number is defined without package id either in the `export()` method or in the pipeline constructor.

Restrictions: 
- You can not overwrite a previously saved package with the same `package id` if the `package id` is explicitly assigned in the `export()` method 
- Packages generated with older versions of AI SDK (without `package id`) will be overwritten
- If a new `package id` is assigned to an existing version of the package it will overwrite the old one
- If no previous version of a package (with a generated or explicitly assigned `package id`) found, AI SDK will assign the version `1` to the created package
- Version defined in the `export()` method takes precedence over the version assigned on constructor level

Now you are ready to bring your model to the shopfloor by building the edge configuration package from the pipeline configuration. You can achieve this by the following step.

In [ ]:
edge_package_path = pipeline.export(destination = '../packages')
edge_package_path

### Test the edge configuration package locally

We suggest you test your edge configuration package on your computer before deploying it to an Edge device. It is possible to do so using notebook [40-TestPipelineLocally.ipynb](40-TestPipelineLocally.ipynb).

### Test the edge configuration package in AI Inference Server

If you also have dockerized Edge environment on your computer, why not try your package with how-to guide [22-test-on-edge-device](../../../howto-guides/22-test-on-edge-device.md)?  
This test can most accurately prove the validity of your package.